<a href="https://colab.research.google.com/github/detektor777/colab_list/blob/main/Real_ESRGAN_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Video

Перейити в гугл диск и загрузить в корень диска файл с видео

https://drive.google.com/drive/ 

In [ ]:
#@title ##**Имя видео файла** { display-mode: "form" }
file_name = input("Введите имя файла и нажимете enter: ") or "input.mp4"

In [ ]:
#@title ##**Клонирование репозитория** { display-mode: "form" }
!nvidia-smi
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

In [ ]:
#@title ##**Подключение к гугл диску** { display-mode: "form" }
from google.colab import drive
drive.mount('/mnt/gdrive')


In [ ]:
#@title ##**Создать папкки для кадров** { display-mode: "form" }
import os
from google.colab import files
import shutil
import os
import shutil, sys 

upload_folder = 'upload'
result_folder = 'results'

if not os.path.exists(upload_folder):
    os.makedirs(upload_folder)

if not os.path.exists(result_folder):
    os.makedirs(result_folder)


In [ ]:
#@title ##**Очистить входящую папку** { display-mode: "form" }
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder)

In [ ]:
#@title ##**Очистить исходящую папку** { display-mode: "form" }
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

In [ ]:
#@title ##**Секвенция видео** { display-mode: "form" }
#если нужно продолжить предедущую обработку, то пропустите этот пункт
import cv2
import imageio
import os
import tqdm
import subprocess
import numpy as np
import time


library = "imageio" #@param ["cv2","pyav","imageio","ffmpeg","skvideo","scipy","moviepy"]
delay = "0.05" #@param [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
if (library == "cv2"):
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    cmd = 'ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 ' + full_path
    output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

    cmd = 'ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of csv=p=0 ' + full_path
    output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()
    vidcap = cv2.VideoCapture(full_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    vidcap = cv2.VideoCapture(full_path)
    success, image = vidcap.read()
    
    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_cv2 = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    while success:
        try:
            cv2.imwrite(f"upload/frame{int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)):09d}.jpg", image)
        except cv2.error:
            print("Error writing to disk. Retrying...")
            continue
        pbar_cv2.update(1)
        time.sleep(float(delay))
        success, image = vidcap.read()

    pbar_cv2.close()
    vidcap.release()

elif (library == "pyav"):
    !pip install av
    import av
    video = av.open(full_path)

    fps_value = video.streams.video[0].average_rate
    frame_count = video.streams.video[0].frames

    if fps_value is not None:
        duration = frame_count / fps_value

        print("FPS: ", fps_value)
        print("Duration: ", duration)
        print("Frames: ", frame_count)

        tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
        pbar_pyav = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
        for packet in video.demux():
            for frame in packet.decode():
                img = frame.to_ndarray(format='rgb24')
                img = img.astype('uint8')  # преобразование к типу uint8
                img_path = f"upload/frame{packet.pts:09d}.jpg"
                imageio.imwrite(img_path, img)
                pbar_pyav.update(1)
                time.sleep(float(delay))

        pbar_pyav.close()
        video.close()
    else:
        print("Error: FPS value is None.")


elif (library == "imageio"):
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    video = imageio.get_reader(full_path)

    fps = video.get_meta_data()['fps']
    frame_count = video.count_frames()
    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_imageio = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video):
      img_path = f"upload/frame{i+1:09d}.jpg"
      if os.path.isfile(img_path):
          pbar_imageio.update(1)
          continue
      while True:
          try:
              imageio.imwrite(f"upload/frame{i:09d}.jpg", frame)
          except Exception as e:
              print(f"Error writing to disk: {str(e)}. Retrying...")
              continue
          break
      pbar_imageio.update(1)
      time.sleep(float(delay))

    pbar_imageio.close()
    video.close()

elif (library == "ffmpeg"):
    !pip install ffmpeg-python
    import ffmpeg
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    probe = ffmpeg.probe(full_path)
    video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
    fps = video_info['r_frame_rate']
    duration = float(video_info['duration'])
    frame_count = int(video_info['nb_frames'])

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    pbar_ffmpeg = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    process = (
        ffmpeg
        .input(full_path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24', qscale=0)
        .run_async(pipe_stdout=True)
    )

    for i in range(frame_count):
        try:
            raw_video = process.stdout.read(video_info['width'] * video_info['height'] * 3)
            frame = np.frombuffer(raw_video, dtype='uint8').reshape((video_info['height'], video_info['width'], 3))
            frame_path = f"upload/frame{i:09d}.jpg"
            imageio.imwrite(frame_path, frame)
        except Exception as e:
            print(f"Error writing to disk: {str(e)}. Retrying...")
            continue
        pbar_ffmpeg.update(1)
        time.sleep(float(delay))

    pbar_ffmpeg.close()
    process.wait()
elif (library == "skvideo"):
    !pip install scikit-video
    import skvideo.io
    path = '/mnt/gdrive/MyDrive/'
    full_path = os.path.join(path, file_name)

    video = skvideo.io.vread(full_path)

    fps_str = skvideo.io.ffprobe(full_path)['video']['@avg_frame_rate']
    fps_parts = fps_str.split('/')
    fps = int(fps_parts[0]) / int(fps_parts[1])
    frame_count = len(video)
    duration = frame_count / fps

    print("FPS: ", fps)
    print("Duration: ", duration)
    print("Frames: ", frame_count)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    outputdict = {'-q:1': '1'}
    pbar_skvideo = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video):
        while True:
            try:
                skvideo.io.vwrite(f"upload/frame{i:09d}.jpg", frame, outputdict=outputdict)
            except Exception as e:
                print(f"Error writing to disk: {str(e)}. Retrying...")
                continue
            break
        pbar_skvideo.update(1)
        time.sleep(float(delay))

    pbar_skvideo.close()

elif (library == "scipy"):
    import scipy.misc
    import scipy.ndimage
    
    full_path = os.path.join(path, file_name)
    video = imageio.get_reader(full_path)
    fps_value = video.get_meta_data()['fps']
    frame_count = video.get_length()

    if fps_value is not None:
        duration = frame_count / fps_value

        print("FPS: ", fps_value)

        tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
        pbar_scipy = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
        for i, im in enumerate(video):
            img_path = f"upload/frame{i+1:09d}.jpg"
            im = Image.fromarray(im)
            im.save(img_path, quality=100)
            pbar_scipy.update(1)
            time.sleep(float(delay))

        pbar_scipy.close()
    else:
        print("Error: FPS value is None.")

elif library == "moviepy":
    from moviepy.editor import VideoFileClip
    from PIL import Image
    full_path = os.path.join(path, file_name)
    video = VideoFileClip(full_path)
    fps_value = video.fps
    frame_count = int(video.duration * fps_value)

    print("FPS: ", fps_value)

    tqdm.tqdm.set_lock(tqdm.tqdm.get_lock())
    pbar_moviepy = tqdm.tqdm(total=frame_count, ncols=100, position=0, leave=True)
    for i, frame in enumerate(video.iter_frames()):
        img_path = f"upload/frame{i+1:09d}.jpg"
        im = Image.fromarray(frame)
        im.save(img_path, quality=100)
        pbar_moviepy.update(1)
        time.sleep(float(delay))

    pbar_moviepy.close()

In [ ]:
#@title ##**Проверка на попущенные кадры (опционально)** { display-mode: "form" }
import os

frame_dir = 'upload'
frames = [int(f.split('.')[0].replace('frame', '')) for f in os.listdir(frame_dir) if f.endswith('.jpg')]
min_frame = min(frames)
max_frame = max(frames)
print(min_frame)
print(max_frame)

missing_frames = []
for i in range(min_frame, max_frame+1):
    if i not in frames:
        missing_frames.append(i)

if len(missing_frames) > 0:
    print(f"Missing frames: {missing_frames}")
else:
    print("All frames present")

In [ ]:
#@title ##**Конфигурация** { display-mode: "form" }
model_name = "RealESRGAN_x2plus" #@param ["RealESRGAN_x2plus","RealESRGAN_x4plus","RealESRNet_x4plus","realesr-general-x4v3","RealESRGAN_x4plus_anime_6B","realesr-animevideov3"]
scale = "2" #@param ["2","3","4"]
face_enhance = "0" #@param [0,1]

command = "python inference_realesrgan.py -n {} -i upload --outscale {}".format(model_name, scale)
if face_enhance:
    command += " --face_enhance"

os.system(command)

In [ ]:
#@title ##**Собрать видео из кадров** { display-mode: "form" }
import cv2
import os

output_file = "/mnt/gdrive/MyDrive/output.mp4"

# Получаем информацию о видео
cap = cv2.VideoCapture(full_path)
fps_of_video = int(cap.get(cv2.CAP_PROP_FPS))
cap.release()

# Получаем список имен файлов в директории
img_files = os.listdir(result_folder)
# Сортируем файлы в алфавитном порядке
img_files.sort()

# Получаем размеры первого кадра для создания видео
frame = cv2.imread(os.path.join(result_folder, img_files[0]))
height, width, _ = frame.shape

# Создаем объект writer, который будет писать каждый кадр в выходной видеофайл
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_file, fourcc, fps_of_video, (width, height))

# Проходим по каждому файлу
for img_file in img_files:
    # Считываем изображение из файла
    img_path = os.path.join(result_folder, img_file)
    frame = cv2.imread(img_path)

    # Записываем кадр в видеофайл
    writer.write(frame)

from moviepy.editor import *


# Освобождаем ресурсы и закрываем видеофайл
writer.release()
print('Сборка завершена')

In [ ]:
#@title ##**Добавить звук** { display-mode: "form" }
# Загружаем видео без звука
video = VideoFileClip(output_file)

# Загружаем звуковую дорожку
audio = AudioFileClip(full_path)

# Добавляем звук к видео
video = video.set_audio(audio)

# Сохраняем полученное видео
video.write_videofile('/mnt/gdrive/MyDrive/output_with_audio.mp4')

In [ ]:
#@title ##**Скачать файл** { display-mode: "form" }
from google.colab import files
files.download('/mnt/gdrive/MyDrive/output_with_audio.mp4', codec='libx264', fps=video_clip.fps, preset='medium')